1\. Write a function that converts numbers among the bin, dec, and hex representations (bin<->dec<->hex)

In [1]:
def func(num):
    num_bin=bin(num)
    num_hex=hex(num)
    num_bin_dec=int(num_bin,2)
    num_hex_dec=int(num_hex,16)
    print("Binary to decimal:",num_bin,"->",num_bin_dec)
    print("Decimal to binary:",num,"->",num_bin)
    print("Decimal to hexadecimal",num,"->",num_hex)
    print("Hexadecimal to decimal",num_hex,"->",num_hex_dec)
func(25)

Binary to decimal: 0b11001 -> 25
Decimal to binary: 25 -> 0b11001
Decimal to hexadecimal 25 -> 0x19
Hexadecimal to decimal 0x19 -> 25


2\. Write a function that converts a 32 bit word into a single precision floating point (i.e. interprets the various bits as sign, mantissa and exponent) and check its validity.

In [2]:
def func2(a):
    xfl=1
    exp=0
    man=0
    if a[0]=='0':#To find sign
        xfl=1
    elif a[0]=='1':
        xfl=-1
    else:
        print("It is not valid.")
    
    for i in range(9):#Calculating exponent
        if i==0:
            exp+=0
        elif a[i]=='1':
            exp+=2**(8-i)
    
    for j in range(23):#Calculate after point
        if a[j+9]=='1':
            man+=2**(-j-1)
    man+=1
    xfl=xfl*man*2**(exp-127)
    print(xfl)
func2('11000000101100000000000000000000')

-5.5


3\. Write a program to determine the underflow and overflow limits (within a factor of 2) for python on your computer. 

**Tip**: define two variables initialized to 1 and halve/double them for a sufficient amount of times to exceed the under/over-flow limits  

In [3]:
def func3(x):
    u=1
    o=1
    for i in range(x):
        u/=2
        o*=2
    print("For ",x,"underflow=",u,"overflow=",o)
func3(4)

For  4 underflow= 0.0625 overflow= 16


4\. Write a program to determine the machine precision

**Tip**: define a new variable by adding an increasingly smaller value (in the same way as the previous problem) and check when the addition starts to have no effect on the number

In [4]:
def func4(y):
    n=1.0
    
    for i in range(y):
        n/=2
        inc=1+n
        print(inc)
func4(54)
print("After 54, it does not effect on the number.")

1.5
1.25
1.125
1.0625
1.03125
1.015625
1.0078125
1.00390625
1.001953125
1.0009765625
1.00048828125
1.000244140625
1.0001220703125
1.00006103515625
1.000030517578125
1.0000152587890625
1.0000076293945312
1.0000038146972656
1.0000019073486328
1.0000009536743164
1.0000004768371582
1.000000238418579
1.0000001192092896
1.0000000596046448
1.0000000298023224
1.0000000149011612
1.0000000074505806
1.0000000037252903
1.0000000018626451
1.0000000009313226
1.0000000004656613
1.0000000002328306
1.0000000001164153
1.0000000000582077
1.0000000000291038
1.000000000014552
1.000000000007276
1.000000000003638
1.000000000001819
1.0000000000009095
1.0000000000004547
1.0000000000002274
1.0000000000001137
1.0000000000000568
1.0000000000000284
1.0000000000000142
1.000000000000007
1.0000000000000036
1.0000000000000018
1.0000000000000009
1.0000000000000004
1.0000000000000002
1.0
1.0
After 54, it does not effect on the number.


5\. Write a function that takes in input three parameters $a$, $b$ and $c$ and prints out the two solutions to the quadratic equation $ax^2+bx+c=0$ using the standard formula:
$$
x=\frac{-b\pm\sqrt{b^2-4ac}}{2a}
$$

(a) use the program to compute the solution for $a=0.001$, $b=1000$ and $c=0.001$

(b) re-express the standard solution formula by multiplying the numerator and the denominator by $-b\mp\sqrt{b^2-4ac}$ and again find the solution for $a=0.001$, $b=1000$ and $c=0.001$. How does it compare with what has been previously obtained, and why?

(c) write a function that computes the roots of a quadratic equation accurately in all cases

In [5]:
import math
def func5(a,b,c):
    x1=(-b+math.sqrt((b**2)-4*a*c))/(2*a)
    x2=(-b-math.sqrt((b**2)-4*a*c))/(2*a)
    print(x1,x2)
    
def re_func5(a,b,c):
    q=-b+math.sqrt(b**2-4*a*c)
    w=-b-math.sqrt(b**2-4*a*c)
    x1_re=q**2/(q*2*a)
    x2_re=w**2/(w*2*a)
    print(x1_re,x2_re)
def c(a,b,c):
    q=-b+math.sqrt(b**2-4*a*c)
    w=-b-math.sqrt(b**2-4*a*c)
    x1c=q**15/(q**14*2*a)
    x2c=w**15/(w**14*2*a)
    print(x1c,x2c)
print("(a)")#a
func5(0.001,1000,0.001)
print("(b)")#b
re_func5(0.001,1000,0.001)
print("After re-express the formula,result must be changed.It looks there is no difference, but the changing on x1 and x2 are really small.")
print("(c)")#c
c(0.001,1000,0.001)
print("To get more accurate result, multiplied the numerator and denominator many times with  −𝑏∓√𝑏**2−4𝑎𝑐 ")

(a)
-9.999894245993346e-07 -999999.999999
(b)
-9.999894245993346e-07 -999999.999999
After re-express the formula,result must be changed.It looks there is no difference, but the changing on x1 and x2 are really small.
(c)
-9.999894245993346e-07 -999999.9999989999
To get more accurate result, multiplied the numerator and denominator many times with  −𝑏∓√𝑏**2−4𝑎𝑐 


6\. Write a program that implements the function $f(x)=x(x−1)$

(a) Calculate the derivative of the function at the point $x = 1$ using the derivative definition:

$$
\frac{{\rm d}f}{{\rm d}x} = \lim_{\delta\to0} \frac{f(x+\delta)-f(x)}{\delta}
$$

with $\delta = 10^{−2}$. Calculate the true value of the same derivative analytically and compare it with the answer your program gives. The two will not agree perfectly. Why?

(b) Repeat the calculation for $\delta = 10^{−4}, 10^{−6}, 10^{−8}, 10^{−10}, 10^{−12}$ and $10^{−14}$. How does the accuracy scales with $\delta$?

In [6]:
from scipy.misc import derivative
def f(x):
    f=x*(x-1)
    return f
def func6(x,l):
    print((f(x+l)-f(x))/l)
print("(a)")#a
func6(1,10**(-2))
print(derivative(f,1.0,dx=10**-2))
print("Two ways have different calculation methods, so the results are slightly different from each other.")
print("(b)")#b
func6(1,10**(-4))
func6(1,10**(-6))
func6(1,10**(-8))
func6(1,10**(-10))
func6(1,10**(-12))
func6(1,10**(-14))
print("When 𝛿 gets smaller, the result also change a little bit. So the accuracy scales might be increase with 𝛿.")

(a)
1.010000000000001
1.0000000000000009
Two ways have different calculation methods, so the results are slightly different from each other.
(b)
1.0000999999998899
1.0000009999177333
1.0000000039225287
1.000000082840371
1.0000889005833413
0.9992007221626509
When 𝛿 gets smaller, the result also change a little bit. So the accuracy scales might be increase with 𝛿.


7\. Consider the integral of the semicircle of radius 1:
$$
I=\int_{-1}^{1} \sqrt(1-x^2) {\rm d}x
$$
which is known to be $I=\frac{\pi}{2}=1.57079632679...$.

Alternatively we can use the Riemann definition of the integral:
$$
I=\lim_{N\to\infty} \sum_{k=1}^{N} h y_k 
$$

with $h=2/N$ the width of each of the $N$ slices the domain is divided into and where
$y_k$ is the value of the function at the $k-$th slice.

(a) Write a program to compute the integral with $N=100$. How does the result compare to the true value?

(b) How much can $N$ be increased if the computation needs to be run in less than a second? What is the gain in running it for 1 minute? 


In [7]:
import scipy.integrate as integrate
import math
def riemann(f,a,b,N):
    h=(b-a)/N
    sum=0.0
    
    for i in range(N):
        sum+=f((2*a+(2*i+1)*h)/2)#midsum
    return sum*h#integral
f=lambda x:math.sqrt(1-x**2)
print("(a)\nIntegral of f function with Riemann definition:\n",riemann(f,-1,1,100))
print("Integral of f function by using scipy method:\n",integrate.quad(f,-1,1))
print("If we compare both values, it differs 2 digits after the dot.\n(b)")
%timeit riemann(f,-1,1,100)
%timeit riemann(f,-1,1,420000)
print("We can increase N to 420000, so computation timing approach to a second.")
print("The gain in running it for 1 min. is that the result will be more accurate.")

(a)
Integral of f function with Riemann definition:
 1.5712827762297954
Integral of f function by using scipy method:
 (1.5707963267948983, 1.0002354500215915e-09)
If we compare both values, it differs 2 digits after the dot.
(b)
239 µs ± 13.8 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
971 ms ± 58.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
We can increase N to 420000, so computation timing approach to a second.
The gain in running it for 1 min. is that the result will be more accurate.
